In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt


In [ ]:
dir_area  = '/Data/forest_area/'
dir_ctrs  = '/Data/data_ancillary/info_countries/'
dir_grid  = '/Data/ELUC_countries/'
dir_tmp   = '/Data/tmp/'


## Calculation

In [ ]:
#Select time
time_sta = '1991'
time_end = '2020'

#Read ISO codes for countries, IPCC countries, and conversions between ISO alpha-3 codes from IPCC and ISO numeric
fname_ctrs_ISO   = dir_ctrs + 'wrld_cntrs_BLUE_TN_upd.nc'
fname_IPCC_codes = dir_ctrs + 'IPCC_regions.xlsx'
fname_ISO_num    = dir_ctrs + 'iso_codes_alpha_numeric.xlsx'
data_IPCC_codes = pd.read_excel(fname_IPCC_codes, sheet_name='region_classification', header=0, usecols=[0, 1, 3])
data_alph_num   = pd.read_excel(fname_ISO_num, header=0)

#Define sources, sinks, and net
selections = ['primary', 'secondary']

#Loop over sources, sinks, and net
for selection in selections:
   
    #Define output file name
    fname_out = dir_area + 'BLUE_forest_area_' + selection + '_countries-ISOcode_vRemapArea_' + time_sta + '-' + time_end + '.xlsx'
    if os.path.exists(fname_out): os.remove(fname_out)

    #Create dicts for storing data
    AREA_ctrs = dict()

    #Create xlsx-file (it will be filled at end of loop with country data from every model)
    with pd.ExcelWriter(fname_out) as writer:

        #Read forest area data
        fname = dir_area + 'BLUE_forest_area_' + selection + '_1991-2020.nc'
        data_area = xr.open_dataset(fname)

        #Select time
        data_area = data_area.sel(time=slice(time_sta, time_end))

        #Read country data
        data_ctrs_ISO = xr.open_dataset(fname_ctrs_ISO)

        #Get lat and lon names
        if 'latitude' in data_area.dims:  lat_name, lon_name = 'latitude', 'longitude'
        else:                             lat_name, lon_name = 'lat', 'lon'

        #Check that model grid and country grid agree
        check_lat = np.max(np.abs(data_area[lat_name].values - data_ctrs_ISO[lat_name].values))
        check_lon = np.max(np.abs(data_area[lon_name].values - data_ctrs_ISO[lon_name].values))
        if check_lat>0.01:  sys.exit('Latitudes do not agree')
        if check_lon>0.01:  sys.exit('Longitudes do not agree')

        #Re-index if there are small deviations in lat and lon
        if (check_lat!=0) or (check_lon!=0):
            print('Country data are re-indexed for ' + model)
            data_ctrs_ISO = data_ctrs_ISO.reindex({lat_name: data_area[lat_name], lon_name: data_area[lon_name]}, method='nearest')       

        #Loop over all country codes
        for i, iso_alpha3 in enumerate(data_IPCC_codes['ISO']):

            if np.mod(i,20)==0:
                print('  -run ' + str(i+1) + ' of ' + str(len(data_IPCC_codes['ISO'])))

            #Get numbeic ISO code of country
            iso_numeric = data_alph_num['Numeric'][data_alph_num['Alpha-3 code']==iso_alpha3].values[0]

            #Select country in country mask
            mask_ISO = data_ctrs_ISO.ISOcode==iso_numeric

            #Get forest area sum in selected country
            data_sel = data_area.where(mask_ISO).sum((lat_name, lon_name))

            #Save values in dict
            AREA_ctrs[iso_alpha3] = data_sel.forest_area.values

        #Convert data to data frame (and sort by country name)
        AREA_ctrs_df = pd.DataFrame(AREA_ctrs, index=data_area.time)
        AREA_ctrs_df = AREA_ctrs_df.reindex(sorted(AREA_ctrs_df.columns), axis=1)

        #Adde units in first cell
        AREA_ctrs_df = AREA_ctrs_df.rename_axis('unit: Tg C/year')

        #Create sheet in xlsx for every model and store country data
        AREA_ctrs_df.to_excel(writer, sheet_name=selection + '_forest', index=True, header=True, float_format='%.6f')
